# Multi objective optimization post-processing analysis

In [ ]:
#import WarmupFUSE
using FUSE
using Plots

In [ ]:
path= "optimization_runs"
path= "/mnt/beegfs/users/meneghini/optimization_runs_supersimple_sms"
path= "/mnt/beegfs/users/meneghini/optimization_runs_supersimple_NSGA2"
path= "/mnt/beegfs/users/meneghini/optimization_runs_simple_NSGA2"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta"
#path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm"
#path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_noonetime"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_slvv"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo"
#path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo_SMSEMOA"

In [ ]:
IMAS.update_ExtractFunctionsLibrary!()
# use this to regenerate CSV cache file
dirs = filter(isdir,sort(readdir(path; join=true)))
println(length(dirs))
dirs = filter(x->!isfile(joinpath(x,"error.txt")) && isfile(joinpath(x,"dd.json")),dirs)#[1:100]
println(length(dirs))
outputs=FUSE.extract(dirs, filter_invalid=:cols);

# cache extrated information to CSV file
import DelimitedFiles
DelimitedFiles.writedlm(joinpath(path,"extract.csv"), Iterators.flatten(([names(outputs)], eachrow(outputs))), ',')

# error analysis
dirs = filter(isdir,sort(readdir(path; join=true)))
errors=FUSE.categorize_errors(dirs;do_plot=true,show_first_line=false)

In [ ]:
# use this to load CSV cache file
outputs=FUSE.DataFrames.DataFrame(FUSE.CSV.File(joinpath(path,"extract.csv")));
println(length(outputs[:,1]))

In [ ]:
# this is just to list the fields that can be queried
IMAS.ExtractFunctionsLibrary

In [ ]:
N=length(outputs[:,"Pelectric_net"])
X=LinRange(1,N,41)
histogram2d(outputs[:,"Pelectric_net"],bins=(X,LinRange(-0,500,50)),ylabel="Pelectric net", xlabel="Generation")
display(hline!([200.0], ls=:dash, label=""))

histogram2d(outputs[:,"ip_ohm"],bins=(X,LinRange(-5,5,50)),ylabel="Ip ohmic", xlabel="Generation")
display(hline!([0.0], ls=:dash, label=""))

In [ ]:
for name in names(outputs)
    println((name,typeof(outputs[1,name])))
    if !(typeof(outputs[1,name]) <: Number) || all(isnan.(outputs[:,name]))
        continue
    end
    display(histogram(outputs[:,name],label="",size=(200,100),linecolor=nothing))
end

In [ ]:
# x axis
xname="capital_cost"; x=outputs[:,xname]
#xname="βpol"; x=outputs[:,xname]

# y axis
yname="βn"; y=outputs[:,yname]
#yname="ip_bs/ip_bs_aux_ohm"; y=outputs[:,"ip_bs"]./outputs[:,"ip_bs_aux_ohm"];

# z axis
#zname="R0"; z=outputs[:,zname]

# color
cname="generation"; c=1:length(x);
#cname="<zeff>"; c=outputs[:,"<zeff>"];
#cname="R0 [m]"; c=outputs[:,"R0"];
#cname="B0 [T]"; c=outputs[:,"B0"];
#cname="ip_ohm"; c=outputs[:,cname]
#cname="ip_bs_aux_ohm/ip"; c=outputs[:,"ip_bs_aux_ohm"]./outputs[:,"ip"];
clim=(-Inf,Inf)

# selection
sname0="Pelectric_net"; s0=outputs[:,sname0]
sname1="ip_ohm"; s1=outputs[:,sname1]
sname2="B0"; s2=outputs[:,sname2]

# subselection criterion
Δf=20.0
n=length(x)
index=1:n
index=findall( (s0.>(200-Δf)) .&& (s0.<(200+Δf)) .&& (abs.(s1).<1) .&& (index.> 1000))
#index=findall( (s0.>(200-n)) .&& (s0.<(200+n)) .&& (index.>0000))
#index=findall( (s0.>(200-n)) .&& (s0.<(200+n)))
#index=findall( (index.>n-1000))
annot=map(x->(x, :center, 3, "courier"), index)

# plotting
scatter(x[index], y[index], marker_z=c[index] ,xlabel=xname, ylabel=yname, colorbar_title=cname, marker=:circle,
    markersize=5, markerstrokewidth=0, label="", clim=clim, alpha=0.5,
#    series_annotations=annot,
    xlim=(0,Inf), ylim=(0,Inf))
#    xlim=(3,15), ylim=(1,3.5))
#    xlim=(0,10), ylim=(0,3))

#plot!(x[index], 0.67.*outputs[:,"βpol"])
#plot!(x[index], 0.75.*outputs[:,"βpol"])

In [ ]:
n=length(dirs)
#n=7864
println(dirs[n])
dd,ini,act=FUSE.load(dirs[n])
FUSE.digest(dd)
#ini.core_profiles

In [ ]:
for cname in names(outputs)
    if all(isnan.(outputs[:,cname]))
        continue
    end
    # color
    c=outputs[:,cname]
    clim=(-Inf,Inf)

    # plotting
    println(cname)
    display(scatter(x[index], y[index], marker_z=c[index] ,xlabel=xname, ylabel=yname, colorbar_title=cname, marker=:circle,
        markersize=5, markerstrokewidth=0, label="", clim=clim, alpha=0.5))
end